In [108]:
import pandas as pd
import pandera as pa
import io
from minio import Minio
import os

ACCESS_ROOT = os.environ.get("PREFECT_MINIO_ACCESS_ROOT")
SECRET_ROOT = os.environ.get("PREFECT_MINIO_SECRET_ROOT")
MINIO_HOST = os.environ.get("PREFECT_MINIO_HOST")
minio_client = Minio(MINIO_HOST, access_key=ACCESS_ROOT,
                     secret_key=SECRET_ROOT, secure=False)
# Query parquet file from MinIO
data = minio_client.get_object(
    "landing-zone", "Recintos_Almendros_Cercanos_y_Otros_Cultivos.xlsx").read()
df = pd.read_excel(io.BytesIO(data), engine="openpyxl",
                       sheet_name="Tratamientos", na_values=[''])




In [109]:
df.dtypes

MovimientoCosecha                       int64
MovimientoFechaDeInicio                object
Producto                               object
ProductoNombre                         object
Formulado                              object
TratamientosPlagaEfectosEnPlagasId      int64
EfectosEnPlagas                        object
TratamientosPlagaMalasHierbasId       float64
SecUserNombre                          object
SecUserNIF                             object
SecUserId                               int64
ParcelaProvinciaId                      int64
ParcelaMunicipioId                      int64
ParcelaPoligono                         int64
Parcela                                 int64
ParcelaRecinto                         object
ParcelaParaje                          object
ParcelaAgregado                        object
ParcelaZona                             int64
ParcelaCosechaCodigoPAC                 int64
ParcelaCosechaCultivoPAC               object
Caldo                             

In [110]:
schema = pa.DataFrameSchema({
        "MovimientoCosecha": pa.Column(pa.Int),
        "MovimientoFechaDeInicio": pa.Column(pa.DateTime),
        "Producto": pa.Column(pa.String, nullable=True),
        "ProductoNombre": pa.Column(pa.String),
        "Formulado": pa.Column(pa.String, nullable=True),
        "TratamientosPlagaEfectosEnPlagasId": pa.Column(pa.String, nullable=True),
        "EfectosEnPlagas": pa.Column(pa.String),
        "TratamientosPlagaMalasHierbasId": pa.Column(pa.String, nullable=True),
        "SecUserNombre": pa.Column(pa.String),
        "SecUserNIF": pa.Column(pa.String, nullable=True),
        "SecUserId": pa.Column(pa.String),
        "ParcelaProvinciaId": pa.Column(pa.String),
        "ParcelaMunicipioId": pa.Column(pa.String),
        "ParcelaPoligono": pa.Column(pa.String),
        "Parcela": pa.Column(pa.String),
        "ParcelaRecinto": pa.Column(pa.String),
        "ParcelaParaje": pa.Column(pa.String, nullable=True),
        "ParcelaAgregado": pa.Column(pa.String),
        "ParcelaZona": pa.Column(pa.String),
        "ParcelaCosechaCodigoPAC": pa.Column(pa.String),
        "ParcelaCosechaCultivoPAC": pa.Column(pa.String),
        "Caldo": pa.Column(pa.String, nullable=True),
        "TipoDeDosisId": pa.Column(pa.String, nullable=True),
        "TipoDeDosisDetalle": pa.Column(pa.String),
        "MovimientoParcelaSuperficieTratada": pa.Column(pa.Float),
        "Cantidad": pa.Column(pa.Float),
        "MovimientoPlazoDeSeguridad": pa.Column(pa.String, nullable=True),
        "MovimientoDosis": pa.Column(pa.Float),
        "ParcelaSuperficieCultivo": pa.Column(pa.Float),
        "ParcelaSuperficieSIGPAC": pa.Column(pa.Float, nullable=True),
        "ParcelaZonaVulnerable": pa.Column(pa.String, nullable=True),
        "UsoDeParcelasId": pa.Column(pa.String, nullable=True),
    }, coerce=True, unique_column_names=True)

validated_df = schema.validate(df)
validated_df.dtypes

MovimientoCosecha                              int64
MovimientoFechaDeInicio               datetime64[ns]
Producto                                      object
ProductoNombre                                object
Formulado                                     object
TratamientosPlagaEfectosEnPlagasId            object
EfectosEnPlagas                               object
TratamientosPlagaMalasHierbasId               object
SecUserNombre                                 object
SecUserNIF                                    object
SecUserId                                     object
ParcelaProvinciaId                            object
ParcelaMunicipioId                            object
ParcelaPoligono                               object
Parcela                                       object
ParcelaRecinto                                object
ParcelaParaje                                 object
ParcelaAgregado                               object
ParcelaZona                                   

In [111]:

validated_df.columns = ["harvestYear", "harvestInitDate", "phytosanitaryId", "phytosanitaryName", "phytosanitaryFormula", "plagueTreatmentEffectsId", "plagueEffects", "plagueTreatmentWeedsId", "secUserName", "secUserNIF", "secUserId", "parcelProvinceId", "parcelMunicipalityId",
                    "parcelPolygonId", "parcelId", "parcelEnclosureId", "parcelGeographicSpot", "parcelAggregatedId", "parcelZoneId", "parcelHarvestPACCode", "parcelHavestPACCropTree", "broth", "doseKind", "doseUnit", "treatedArea", "phytosanitaryQuantityMovement", "safePeriodMovement", "doseMovement", "parcelArea", "parcelAreaSIGPAC", "parcelVulnerableArea", "parcelSIGPACCode"]
# Hide sensitive data
validated_df = validated_df.drop(columns=["secUserNIF"])
# NOTE: Thanks to Jupyter Notebook, I found out that some number columns are being read as objects
# Trim spaces and tabs to all object columns
validated_df = validated_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# Convert NULL, NP, NaN, etc. to None
validated_df = validated_df.replace(
    {pd.NA: None, "NP": None, "NaN": None, "": None, "NULL": None})
# Convert strings to uppercase
validated_df["secUserName"] = validated_df["secUserName"].str.upper()
# Get data year
data_year = validated_df['harvestYear'].iloc[:1].values[0]

In [112]:
validated_df.dtypes

harvestYear                               int64
harvestInitDate                  datetime64[ns]
phytosanitaryId                          object
phytosanitaryName                        object
phytosanitaryFormula                     object
plagueTreatmentEffectsId                 object
plagueEffects                            object
plagueTreatmentWeedsId                  float64
secUserName                              object
secUserId                                object
parcelProvinceId                         object
parcelMunicipalityId                     object
parcelPolygonId                          object
parcelId                                 object
parcelEnclosureId                        object
parcelGeographicSpot                     object
parcelAggregatedId                       object
parcelZoneId                             object
parcelHarvestPACCode                     object
parcelHavestPACCropTree                  object
broth                                   

In [113]:
data = validated_df.to_parquet()
# To dataframe
df = pd.read_parquet(io.BytesIO(data))
# Show complete dataframe
pd.set_option('display.max_columns', None)
df.head(10)

,harvestYear,harvestInitDate,phytosanitaryId,phytosanitaryName,phytosanitaryFormula,plagueTreatmentEffectsId,plagueEffects,plagueTreatmentWeedsId,secUserName,secUserId,parcelProvinceId,parcelMunicipalityId,parcelPolygonId,parcelId,parcelEnclosureId,parcelGeographicSpot,parcelAggregatedId,parcelZoneId,parcelHarvestPACCode,parcelHavestPACCropTree,broth,doseKind,doseUnit,treatedArea,phytosanitaryQuantityMovement,safePeriodMovement,doseMovement,parcelArea,parcelAreaSIGPAC,parcelVulnerableArea,parcelSIGPACCode
0,2022,2022-03-16,25361,BERELEX 40 SG,ACIDO GIBERELICO 40% [SG] P/P,33000001001156,AUMENTO DE CUAJADO,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,2,204,1,MEDIA HUERTA,0,0,108,PERAL,1000.0,1,g/Hl,0.59,0.01475,None,2.5,0.59,0.59,1,FY
1,2022,2022-03-16,25361,BERELEX 40 SG,ACIDO GIBERELICO 40% [SG] P/P,33000001001156,AUMENTO DE CUAJADO,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,2,205,1,MEDIA HUERTA,0,0,108,PERAL,1000.0,1,g/Hl,0.31,0.00775,None,2.5,0.31,0.31,1,FY
2,2022,2022-03-16,25361,BERELEX 40 SG,ACIDO GIBERELICO 40% [SG] P/P,33000001001156,AUMENTO DE CUAJADO,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,2,206,1,MEDIA HUERTA,0,0,108,PERAL,1000.0,1,g/Hl,0.54,0.01350,None,2.5,0.54,0.54,1,FY
3,2022,2022-03-21,25361,BERELEX 40 SG,ACIDO GIBERELICO 40% [SG] P/P,33000001001156,AUMENTO DE CUAJADO,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,2,204,1,MEDIA HUERTA,0,0,108,PERAL,1000.0,1,g/Hl,0.59,0.01475,None,2.5,0.59,0.59,1,FY
4,2022,2022-03-21,25361,BERELEX 40 SG,ACIDO GIBERELICO 40% [SG] P/P,33000001001156,AUMENTO DE CUAJADO,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,2,205,1,MEDIA HUERTA,0,0,108,PERAL,1000.0,1,g/Hl,0.31,0.00775,None,2.5,0.31,0.31,1,FY
5,2022,2022-03-21,25361,BERELEX 40 SG,ACIDO GIBERELICO 40% [SG] P/P,33000001001156,AUMENTO DE CUAJADO,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,2,206,1,MEDIA HUERTA,0,0,108,PERAL,1000.0,1,g/Hl,0.54,0.01350,None,2.5,0.54,0.54,1,FY
6,2022,2022-04-25,24526,TEPPEKI,FLONICAMID 50% [WG] P/P,252,PULGONES,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,1,35,2,EL CAUDO,0,0,109,MANZANO,1000.0,1,g/Hl,0.38,0.05320,21,14.0,0.38,0.48,1,FY
7,2022,2022-04-25,24526,TEPPEKI,FLONICAMID 50% [WG] P/P,252,PULGONES,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,1,35,2,None,0,0,109,MANZANO,1000.0,1,g/Hl,0.10,0.01400,21,14.0,0.10,0.48,1,FY
8,2022,2022-04-25,24526,TEPPEKI,FLONICAMID 50% [WG] P/P,252,PULGONES,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,1,35,5,EL CAUDO,0,0,109,MANZANO,1000.0,1,g/Hl,0.09,0.01260,21,14.0,0.09,0.09,0,TA
9,2022,2022-04-25,24526,TEPPEKI,FLONICAMID 50% [WG] P/P,252,PULGONES,NaN,"CARVAJAL MORALA, JOSE IGNACIO",901046400000000010,50,99,1,36,1,EL CAUDO,0,0,109,MANZANO,1000.0,1,g/Hl,0.58,0.08120,21,14.0,0.58,0.58,1,FY
